In [37]:
import tensorflow as tf
import numpy as np
import pandas as pd

# Tensorboard

In [82]:
df = pd.read_csv('./data/data_deep_nn_2.csv', header=None, delimiter=',', dtype='float32')
df.values

array([[ 0.,  0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  1.,  0.],
       [ 1.,  1.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  1.],
       [ 0.,  1.,  1.,  0.,  0.]], dtype=float32)

In [83]:
x = df.ix[:,0:1]

In [84]:
x.values

array([[ 0.,  0.],
       [ 1.,  0.],
       [ 1.,  1.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  1.],
       [ 0.,  1.]], dtype=float32)

In [85]:
y = df.ix[:,2:]

In [86]:
y.values

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.]], dtype=float32)

### NN 모델 만들기

기존과 다른 점은 각각의 레이어별로 name_scope()을 이용해 묶어 주는 것 뿐이다. 텐서보드에서 하나의 덩어리로 보기 위해서이다. 

* tf.name_scope(hidden_1)
* tf.name_scope(hidden_2)
* tf.name_scope(output)

In [87]:
x_true = tf.placeholder(tf.float32, name='x_true')
y_true = tf.placeholder(tf.float32, name='y_true')

In [116]:
with tf.name_scope('hidden_1'):
    weight_1 = tf.Variable(tf.random_uniform([2,10], -1., 1.), name='weight_1')
    bias_1 = tf.Variable(tf.zeros([10]), dtype=tf.float32, name='bias_1')
    y_hidden_1 = tf.nn.relu(tf.add(tf.matmul(x_true, weight_1), bias_1))

In [117]:
with tf.name_scope('hidden_2'):
    weight_2 = tf.Variable(tf.random_uniform([10,20], -1., 1.), name='weight_2')
    bias_2 = tf.Variable(tf.zeros([20]), dtype=tf.float32, name='bias_2')
    y_hidden_2 = tf.nn.relu(tf.add(tf.matmul(y_hidden_1, weight_2), bias_2))

In [118]:
with tf.name_scope('output'):
    weight_3 = tf.Variable(tf.random_normal([20,3], -1., 1.), name='weight_3')
    bias_3 = tf.Variable(tf.zeros([3]), dtype=tf.float32, name='bias_3')
    y_pred = tf.nn.relu(tf.add(tf.matmul(y_hidden_2, weight_3), bias_3))

In [119]:
with tf.name_scope('optimizer'):
    cost = tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true)
    cost = tf.reduce_mean(cost)
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.1)
    optimizer = optimizer.minimize(cost)
    
    tf.summary.scalar('cost', cost)

위에서와 같이 tf.summary.scalar를 이용해서 수집하고 싶은 값들을 지정할 수 있습니다.

### 학습시키기

In [120]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

텐서보드에서 표시할 값들을 수집합니다.

In [121]:
merged = tf.summary.merge_all()

로그를 저장할 핸들러를 설정합니다.

In [122]:
writer = tf.summary.FileWriter('./logs', sess.graph)

최종적으로 tensorboard --logdir=./logs (--port 6006) 쳐주면 텐서보드가 실행됩니다. 

In [126]:
for step in xrange(1000):
    sess.run(optimizer, feed_dict={x_true:x, y_true:y})
    summary = sess.run(merged, feed_dict={x_true:x, y_true:y}) # Why error?????
    writer.add_summary(summary, step)

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_2' with dtype float
	 [[Node: Placeholder_2 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'Placeholder_2', defined at:
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/comsy/code/tensorflow_python2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/comsy/code/tensorflow_python2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/comsy/code/tensorflow_python2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.

### 평가하기 

In [128]:
prediction = tf.argmax(y_pred, 1)
target = tf.argmax(y_true, 1)

In [129]:
print "예측값:", sess.run(prediction, {x_true:x})
print "실제값:", sess.run(target, {y_true:y})

예측값: [0 0 0 0 0 0 0]
실제값: [0 1 2 0 0 2 0]


In [131]:
correct_prediction = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print "정확도: %.2f" % sess.run(accuracy * 100, {x_true: x, y_true: y})

정확도: 57.14
